In [6]:
#  Credit to Moshe Zadka for this code from an Article on Medium
# a really nice intro into ipywidgets,  tasks, reactor and simplicity of subprocesser running for Git integration

from twisted.internet import asyncioreactor
#asyncioreactor.install()
from twisted.internet import reactor, task 
import ipywidgets , datetime , subprocess, functools, os 


#LoopingCall can only be exited by an exception so define one 
class DoneError(Exception): 
    pass
saveCount = 0
def time_out_counter(reactor):
    label = ipywidgets.Label("Time left: 1:00")
    current_seconds = datetime.timedelta(minutes=1).total_seconds()
    def decrement(count):
        nonlocal current_seconds
        current_seconds -= count 
        time_left = datetime.timedelta(seconds=max(current_seconds,0))
        minutes, left = divmod(time_left, minute)
        seconds = int(left.total_seconds())
        label.value = f"Time left: {minutes}:{seconds:02}"
        if current_seconds < 0 :
            print("Times up")
            raise DoneError("finished")  
    minute = datetime.timedelta(minutes=1)         
    # the withCount makes available the number of calls which should have occurred since it was last invoked.
    call = task.LoopingCall.withCount(decrement)
    call.reactor=reactor
    d = call.start(1)
    d.addErrback(lambda f:f.trap(DoneError))
    return d, label

def editor(fname):
    #use continuous_update = false so that it only saves when textarea loses focus
    textarea = ipywidgets.Textarea(continuous_update = False)
    textarea.rows = 20
    # ouput widget can capture and display stdout, stderr , or any other text
    output = ipywidgets.Output()
    runner = functools.partial(subprocess.run , capture_output=True , text=True,  check = True )
    def save(_ignored):
        global saveCount 
        output.clear_output()
        saveCount += 1
        with output:
            with open(fname, "w") as fpout : 
                 fpout.write(textarea.value)
            print(f"Saving...{saveCount}" , end='')
            # push to git and capture any errors via output
            try: 
                print(f"\nAdding file to Git {fname}")
                runner(["git", "add", fname])
                runner(["git", "commit", "-m",f"updated {fname}"])
                runner(["git", "push"])
                print(f"\nCommit complete {saveCount}")
            except subprocess.CalledProcessError as exc:
                print("Could not send")
                print(exc.stdout)
                print(exc.stderr)
            else:
                print("Git push Done ")
    textarea.observe(save, names ="value")
    return textarea, output, save
    
def journal():
        date = str(datetime.date.today())
        title = f"Log: start date {date}"
        filename = os.path.join( f"Journal-{date}.txt")
        print( reactor.getReaders())
        d, clock = time_out_counter(reactor)
        print(f"reactor running: {reactor.running} ")
        textarea , output , save = editor(filename)
        box = ipywidgets.VBox([ipywidgets.Label(title), textarea, clock, output])
        d.addCallback(save)
        return box
    
print(f"running ? {reactor.running}")
if (not reactor.running):
    try:
        reactor.run()

    except Exception as e:
        print(e)
        import sys
        del sys.modules['twisted.internet.reactor']
        from twisted.internet import reactor
        from twisted.internet import asyncioreactor
        asyncioreactor.install()

print(F"Now? : {reactor.running}")
if (not reactor.running):
    try:
        reactor.run()
    except Exception as e:
        print(e)
        
print(F"Finally Now? : {reactor.running}")

running ? False
This event loop is already running
Now? : True
Finally Now? : True


In [7]:
journal()


reactor running: True 
